# Clojupyter + oz example

This notebook demonstrates basic usage of oz from a Jupyter notebook using the Clojuypyter kernel.

Note that the visualizations below may not render if you're looking at this on GitHub, but will render if you load the notebook into a running Jupyter instance.

In [2]:
(require '[clojupyter.misc.helper :as helper])
(helper/add-dependencies '[metasoarous/oz "1.5.0"])

{[clj-time "0.11.0"] #{[joda-time "2.8.2"]}, [org.apache.httpcomponents/httpclient "4.5.5" :exclusions [[org.clojure/clojure]]] #{[commons-logging "1.2"]}, [org.flatland/ordered "1.5.2"] #{[org.flatland/useful "0.9.0"]}, [org.clojure/tools.logging "0.4.1" :exclusions [[org.clojure/clojure]]] nil, [ring/ring-core "1.7.1"] #{[commons-fileupload "1.3.3"] [crypto-equality "1.0.0"] [crypto-random "1.2.0"]}, [reagent "0.8.1"] #{[cljsjs/react-dom-server "16.3.2-0"] [cljsjs/create-react-class "15.6.3-0"] [cljsjs/react-dom "16.3.2-0"] [cljsjs/react "16.3.2-0"]}, [com.taoensso/timbre "4.10.0"] #{[io.aviso/pretty "0.1.33"]}, [commons-io "2.6" :exclusions [[org.clojure/clojure]]] nil, [org.msgpack/msgpack "0.6.12"] #{[com.googlecode.json-simple/json-simple "1.1.1" :exclusions [[junit]]] [org.javassist/javassist "3.18.1-GA"]}, [org.sonatype.aether/aether-spi "1.13.1"] nil, [com.clojure-goes-fast/lazy-require "0.1.1"] nil, [org.apache.maven.wagon/wagon-http "2.2"] #{[org.apache.maven.wagon/wagon-htt

In [3]:
(helper/add-dependencies '[incanter "1.9.3"])

{[org.bouncycastle/bctsp-jdk14 "1.46"] #{[org.bouncycastle/bcprov-jdk14 "1.46"] [org.bouncycastle/bcmail-jdk14 "1.46"]}, [org.apache.xmlgraphics/batik-css "1.9.1"] nil, [org.clojure/clojure "1.8.0"] nil, [org.apache.poi/poi-ooxml-schemas "3.16"] #{[org.apache.xmlbeans/xmlbeans "2.6.0"]}, [org.apache.xmlgraphics/xmlgraphics-commons "2.2"] #{[commons-io "1.3.1"] [commons-logging "1.0.4"]}, [org.apache.xmlgraphics/batik-constants "1.9.1"] nil, [net.sf.opencsv/opencsv "2.3"] nil, [net.sourceforge.f2j/arpack_combined_all "0.1"] nil, [org.clojure/tools.macro "0.1.5"] nil, [net.mikera/vectorz "0.62.0"] #{[net.mikera/mathz "0.3.0"] [us.bpsm/edn-java "0.4.6"] [net.mikera/randomz "0.3.0"]}, [org.clojure/core.incubator "0.1.4"] nil, [net.sourceforge.parallelcolt/parallelcolt "0.10.1"] #{[com.github.rwl/JKLU "1.0.0"] [net.sourceforge.csparsej/csparsej "1.1.1"] [com.googlecode.netlib-java/netlib-java "0.9.3"] [net.sourceforge.parallelcolt/optimization "1.0"] [net.sourceforge.jtransforms/jtransforms

In [4]:
(helper/add-dependencies '[cheshire "5.12.0"])

{[cheshire "5.12.0"] #{[tigris "0.1.2"] [com.fasterxml.jackson.dataformat/jackson-dataformat-cbor "2.15.2" :exclusions [[com.fasterxml.jackson.core/jackson-databind]]] [com.fasterxml.jackson.dataformat/jackson-dataformat-smile "2.15.2" :exclusions [[com.fasterxml.jackson.core/jackson-databind]]] [com.fasterxml.jackson.core/jackson-core "2.15.2"]}, [com.fasterxml.jackson.core/jackson-core "2.15.2"] nil, [com.fasterxml.jackson.dataformat/jackson-dataformat-smile "2.15.2" :exclusions [[com.fasterxml.jackson.core/jackson-databind]]] nil, [com.fasterxml.jackson.dataformat/jackson-dataformat-cbor "2.15.2" :exclusions [[com.fasterxml.jackson.core/jackson-databind]]] nil, [tigris "0.1.2"] nil}

In [5]:
(require '[oz.notebook.clojupyter :as oz]
         '[cheshire.core :as json]
         '[clojure.java.io :as io]
         '[incanter.stats :as stats]
         '[clojure.walk :as walk])

nil

## Utility functions

Given a JSON file, we transform it into a Clojure map data structure to manipulate the data inside the JSON.

In [6]:
(defn json->map [file-name]
 (->> file-name
      io/reader
      json/parse-stream
      walk/keywordize-keys))

#'user/json->map

Given an array of data, we extract the basic statistics from them. The statistics generated are: mean, standard deviation, variance, median and a basic confidence interval.

In [7]:
(defn simple-stats [data]
  {:ci       (stats/simple-ci data)
   :mean     (stats/mean data)
   :sd       (stats/sd data)
   :variance (stats/variance data)
   :median   (stats/median data)})

#'user/simple-stats

Given a map data structure, we write it to a file. The data is formatted to be pretty printed in the file.

In [8]:
(defn map->json
  ([file-output data] (map->json file-output data true))
  ([file-output data pretty]
   (with-open [writer (io/writer file-output)]
     (json/generate-stream data writer {:pretty pretty}))))

#'user/map->json

In [9]:
(defn get-stats [cores map-algo-filtered]
  (map (fn [core]
       (let [core-int (-> core name Integer/parseInt)]
         (-> map-algo-filtered
             (get core)
             simple-stats
             (assoc :core core-int))))
     cores))

#'user/get-stats

In [10]:
(defn analyze-entries [type cores path]
  (let [proc-nums  (map #(keyword (str %)) (range 1 (inc cores)))
        data       (->> path
                        io/file
                        .listFiles
                        (filter #(.isFile %))
                        (filter #(not (re-matches #"analysis(\-*\w*)*\.json" (.getName %))))
                        (map json->map))
        algorithms (map #(-> % :algorithm keyword) data)
        pre-proc   (reduce (fn [acc entry]
                             (assoc acc
                                    (keyword (:algorithm entry))
                                    (:results entry)))
                           {}
                           data)]
    {:type    type
     :results (map #(identity
                   {:algorithm %
                    :stats     (get-stats proc-nums (get pre-proc %))})
                 algorithms)}))

#'user/analyze-entries

In [11]:
(defn analysis [type cores file-output dir-input]
  (map->json file-output
               (analyze-entries type cores dir-input)
               true))

#'user/analysis

In [12]:
(analysis "counts"
           64
           "data/LLIC/counts/64/analysis.json"
           "data/LLIC/counts/64/")

#object[java.io.BufferedWriter 0x2c6eb03b "java.io.BufferedWriter@2c6eb03b"]